In [59]:
from music21 import *
import os
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

notes = []
def generateNotes():
    for file in os.listdir("MidiFiles"):
        notes1 = []
        file_location = os.path.join("MidiFiles" , file)
        midi1 = converter.parse(file_location)
        notes_to_parse = None
        parts = instrument.partitionByInstrument(midi1)
        if parts:  # file has instrument parts
            notes_to_parse = parts.parts[0].recurse()
        else:  # file has notes in a flat structure
            notes_to_parse = midi1.flat.notes
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append(".".join(str(n) for n in element.normalOrder))
    return notes

In [60]:
def mapInputsToOutputs():
    notes = generateNotes()
    n_vocab = len(set(notes))
    sequence_length = 32
    # get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    network_input1 = []
    network_output = []
    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input1.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    n_patterns = len(network_input1)
    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input1, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)
    network_output = np_utils.to_categorical(network_output)
    return (n_vocab, network_input, network_output, network_input1)

In [61]:
def buildModel():
    (n_vocab, network_input, network_output, network_input1) = mapInputsToOutputs()
    model = Sequential()
    model.add(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

In [62]:
buildModel()

In [5]:
def train():
    """ train the neural network """
    model = buildModel()
    (n_vocab, network_input, network_output, network_input1) = mapInputsToOutputs()
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=200, batch_size=128, callbacks=callbacks_list)
    return model

In [6]:
model = train()

Epoch 1/200
45/45 [==============================] - 7s 37ms/step - loss: 5.1058
Epoch 2/200
45/45 [==============================] - 2s 36ms/step - loss: 4.9729
Epoch 3/200
45/45 [==============================] - 2s 37ms/step - loss: 4.9616
Epoch 4/200
45/45 [==============================] - 2s 36ms/step - loss: 4.9487
Epoch 5/200
45/45 [==============================] - 2s 37ms/step - loss: 4.9464
Epoch 6/200
45/45 [==============================] - 2s 37ms/step - loss: 5.0373
Epoch 7/200
45/45 [==============================] - 2s 36ms/step - loss: 4.9433
Epoch 8/200
45/45 [==============================] - 2s 36ms/step - loss: 4.9434
Epoch 9/200
45/45 [==============================] - 2s 36ms/step - loss: 4.9334
Epoch 10/200
45/45 [==============================] - 2s 37ms/step - loss: 4.9327
Epoch 11/200
45/45 [==============================] - 2s 36ms/step - loss: 4.9392
Epoch 12/200
45/45 [==============================] - 2s 36ms/step - loss: 4.9259
Epoch 13/200
45/45 [=====

45/45 [==============================] - 2s 40ms/step - loss: 4.2844
Epoch 101/200
45/45 [==============================] - 2s 40ms/step - loss: 4.2536
Epoch 102/200
45/45 [==============================] - 2s 39ms/step - loss: 4.2211
Epoch 103/200
45/45 [==============================] - 2s 41ms/step - loss: 4.1995
Epoch 104/200
45/45 [==============================] - 2s 40ms/step - loss: 4.1676
Epoch 105/200
45/45 [==============================] - 2s 41ms/step - loss: 4.1283
Epoch 106/200
45/45 [==============================] - 2s 41ms/step - loss: 4.1199
Epoch 107/200
45/45 [==============================] - 2s 41ms/step - loss: 4.0696
Epoch 108/200
45/45 [==============================] - 2s 41ms/step - loss: 4.0295
Epoch 109/200
45/45 [==============================] - 2s 41ms/step - loss: 4.0144
Epoch 110/200
45/45 [==============================] - 2s 41ms/step - loss: 4.0004
Epoch 111/200
45/45 [==============================] - 2s 41ms/step - loss: 3.9609
Epoch 112/200
45/4

45/45 [==============================] - 2s 40ms/step - loss: 0.6748
Epoch 199/200
45/45 [==============================] - 2s 41ms/step - loss: 0.5989
Epoch 200/200
45/45 [==============================] - 2s 40ms/step - loss: 0.6277


In [63]:
def generation():
    notes = generateNotes()
    (n_vocab, network_input, network_output, network_input1) = mapInputsToOutputs()
    pitchnames = sorted(set(item for item in notes))
    start = numpy.random.randint(0, len(network_input)-1)
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    pattern = network_input1[start]
    prediction_output = []
    # generate 500 notes
    for note_index in range(30):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        prediction = model.predict(prediction_input, verbose=0)
        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
    return prediction_output

In [64]:
prediction_output = generation()

In [65]:
offset = 0
output_notes = []
print(prediction_output)
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5
print(output_notes)

['F#4', '6.7', 'G4', 'F#4', 'F6', 'D4', 'A3', 'B3', '8.11', '3.6', '8.11', '5', 'B3', '8.11.3', 'F#4', '8.11.3', '6.9.11.2', 'A4', 'D5', 'E5', '10.2.5', '10.2', 'B4', 'A4', 'A4', '11.0.4.7', 'F#5', '2.5.7.10', 'B3', 'C#5']
[<music21.note.Note F#>, <music21.chord.Chord F# G>, <music21.note.Note G>, <music21.note.Note F#>, <music21.note.Note F>, <music21.note.Note D>, <music21.note.Note A>, <music21.note.Note B>, <music21.chord.Chord G# B>, <music21.chord.Chord E- F#>, <music21.chord.Chord G# B>, <music21.chord.Chord F>, <music21.note.Note B>, <music21.chord.Chord G# B E->, <music21.note.Note F#>, <music21.chord.Chord G# B E->, <music21.chord.Chord F# A B D>, <music21.note.Note A>, <music21.note.Note D>, <music21.note.Note E>, <music21.chord.Chord B- D F>, <music21.chord.Chord B- D>, <music21.note.Note B>, <music21.note.Note A>, <music21.note.Note A>, <music21.chord.Chord B C E G>, <music21.note.Note F#>, <music21.chord.Chord D F G B->, <music21.note.Note B>, <music21.note.Note C#>]


In [66]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output2.mid')

'test_output2.mid'